In [ ]:
# Import Python API
# https://github.com/google/earthengine-api
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='gbsc-gcp-lab-emordeca')

In [ ]:
# Load in the feature collections
background_2018 = ee.FeatureCollection('users/sagems/background_2018')
pineapple_2018 = ee.FeatureCollection('users/sagems/pineapple_2018')
background_2019 = ee.FeatureCollection('users/sagems/background_2019')
pineapple_2019 = ee.FeatureCollection('users/sagems/pineapple_2019')

# borders for costa rica
# https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0
borders = ee.FeatureCollection('FAO/GAUL/2015/level0')
costa_rica = borders.filter(ee.Filter.eq('ADM0_NAME', 'Costa Rica'))

In [ ]:
# Define a function to mask clouds
# function from https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED
def maskS2clouds(image):
    qa = image.select('QA10')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

In [ ]:
sentinel_2018 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2018-01-01', '2018-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .filterBounds(costa_rica) \
    .median() \
    .select(['B.*', 'TCI.*'])

sentinel_2019 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2019-01-01', '2019-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .filterBounds(costa_rica) \
    .median() \
    .select(['B.*', 'TCI.*'])

In [ ]:
def add_indices(image):
    # Add NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    image = image.addBands(ndvi)

    # Add SAVI
    savi = image.expression(
        '((NIR - Red) / (NIR + Red + 0.5)) * 1.5',
        {
            'NIR': image.select('B8'),
            'Red': image.select('B4')
        }
    ).rename('SAVI')
    image = image.addBands(savi)

    # Add NDMI
    ndmi = image.normalizedDifference(['B8', 'B11']).rename('NDMI')
    image = image.addBands(ndmi)

    # Add NDWI
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    image = image.addBands(ndwi)

    return image

# select bands of interest
bands = ["B2", "B3", "B4", "B8", "B11"]
sentinel_2018 = sentinel_2018.select(bands)
sentinel_2019 = sentinel_2019.select(bands)

# add indices
sentinel_2018 = add_indices(sentinel_2018)
sentinel_2019 = add_indices(sentinel_2019)

print('done')


done


In [ ]:
sampled_data = sentinel_2018.sampleRegions(
    collection= background_2018,
    scale=10,
    geometries = True
)

# Export the sampled data as a table to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=sampled_data,
    description='background_2018_sampled_data',
    folder='GEE_export',
    fileFormat='CSV'
)
task.start()

In [ ]:
# Define the export parameters
export_params = {
    'image': sentinel_2019,
    'description': 'sentinel_2019',  # Set your desired export name
    'scale': 10000,  # Set the desired spatial resolution (e.g., 30 meters per pixel)
    'fileFormat': 'GeoTIFF',  # Specify the file format
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=mapbiomas_sentinel_2019,
    description='mapbiomas_sentinel_2019',
    fileFormat='CSV'
    )
task.start()


In [ ]:
collections_to_export = [mapbiomas_sentinel_2015, mapbiomas_sentinel_2016, mapbiomas_sentinel_2017, mapbiomas_sentinel_2018, mapbiomas_sentinel_2019]
collections_title = ['mapbiomas_sentinel_2015', 'mapbiomas_sentinel_2016', 'mapbiomas_sentinel_2017', 'mapbiomas_sentinel_2018', 'mapbiomas_sentinel_2019']
for i, collection in enumerate(collections_to_export):
  task = ee.batch.Export.table.toDrive(
    collection=collection,
    description=collections_title[i],
    fileFormat='CSV'
    )
  task.start()
